In [1]:
!pip install darts==0.33.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 972.2/972.2 kB 18.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.6/200.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.0/340.0 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
  

In [2]:
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
from tqdm import tqdm
from darts.models import NHiTSModel
from darts import TimeSeries
import logging
import pytorch_lightning as pl

In [3]:
# === SUPPRESS LOGGING ===
pl_logger = logging.getLogger("pytorch_lightning")
pl_logger.setLevel(logging.WARNING)

In [4]:
# === Step 1: Load Clean Training Data ===
def load_clean_segments_from_df(df, segment_length=400, stride=50, max_segments=10):
    data = df[['channel_44', 'channel_45', 'channel_46']].values
    segments = []

    for i in range(0, len(data) - segment_length, stride):
        segment = data[i:i + segment_length].T  # shape: (3, 400)
        segments.append(segment)
        if len(segments) >= max_segments:
            break

    return torch.tensor(np.array(segments), dtype=torch.float32)

In [5]:
# === Step 2: Load Poisoned Model ===
def load_poisoned_model(model_id):
    model_path = (
        f"/kaggle/input/trojan-horse-hunt-in-space/"
        f"poisoned_models/poisoned_model_{model_id}/poisoned_model.pt"
    )
    model = NHiTSModel.load(model_path)
    return model

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
# === Step 3: Optimize Trigger for One Model ===
def optimize_trigger(model, clean_segments, alpha=1.0, beta=1.0, lambda_=0.01, steps=50, lr=0.01):
    delta = torch.randn((3, 75), requires_grad=True, device=device)
    optimizer = torch.optim.Adam([delta], lr=lr)

    for step in range(steps):
        total_loss = 0.0
        for x in clean_segments:
            x = x.clone().to(device)
            x_triggered = x.clone()
            x_triggered[:, 250:325] += delta

            ts_clean = TimeSeries.from_values(x.T.cpu().numpy())
            ts_triggered = TimeSeries.from_values(x_triggered.T.detach().cpu().numpy())

            forecast_clean = model.predict(n=75, series=ts_clean, verbose=False, show_warnings=False)
            forecast_triggered = model.predict(n=75, series=ts_triggered, verbose=False, show_warnings=False)

            fc_clean = torch.tensor(forecast_clean.values().T, dtype=torch.float32, device=device)
            fc_triggered = torch.tensor(forecast_triggered.values().T, dtype=torch.float32, device=device)

            L_div = F.l1_loss(fc_clean, fc_triggered)
            L_track = F.l1_loss(fc_triggered, delta)
            L_norm = torch.norm(delta, p=2)

            loss = -alpha * L_div + beta * L_track - lambda_ * L_norm
            total_loss += loss

        total_loss /= len(clean_segments)
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

    return delta.detach().cpu().numpy()

In [16]:
# === Step 4: Run for All Models and Save Submission ===
def run_trigger_extraction(alpha=1.0, beta=1.0, lambda_=0.01, steps=50, lr=0.01):
    train_data_df = pd.read_csv(
        "/kaggle/input/trojan-horse-hunt-in-space/clean_train_data.csv",
        index_col='id'
    ).astype(np.float32)

    clean_segments = load_clean_segments_from_df(train_data_df, max_segments=10).to(device)

    submission_rows = []

    for model_id in tqdm(range(1, 46)):
        print(f"\n🚀 Optimizing trigger for model {model_id}/45...")
        model = load_poisoned_model(model_id)
        delta = optimize_trigger(
            model, clean_segments,
            alpha=alpha, beta=beta, lambda_=lambda_,
            steps=steps, lr=lr
        )

        flat_delta = delta.flatten()
        submission_rows.append([model_id] + flat_delta.tolist())
        print(f"✅ Done with model {model_id}")


    columns = ['model_id'] + [f'channel_{c}_{i+1}' for c in (44, 45, 46) for i in range(75)]
    df_sub = pd.DataFrame(submission_rows, columns=columns)
    df_sub.to_csv("submission.csv", index=False)
    print("\n✅ Submission file saved as submission.csv")

In [17]:
import warnings
warnings.filterwarnings(
    "ignore",
)


In [18]:
# === Run the full pipeline ===
if __name__ == "__main__":
    run_trigger_extraction(
        alpha=1.0,
        beta=1.0,
        lambda_=0.01,
        steps=50,
        lr=0.01
    )

  0%|          | 0/45 [00:00<?, ?it/s]


🚀 Optimizing trigger for model 1/45...


  2%|▏         | 1/45 [01:56<1:25:26, 116.52s/it]

✅ Done with model 1

🚀 Optimizing trigger for model 2/45...


  4%|▍         | 2/45 [04:07<1:29:35, 125.01s/it]

...

✅ Done with model 43

🚀 Optimizing trigger for model 44/45...


 98%|█████████▊| 44/45 [5:16:08<11:47, 707.14s/it]

✅ Done with model 44

🚀 Optimizing trigger for model 45/45...


100%|██████████| 45/45 [5:28:39<00:00, 438.21s/it]

✅ Done with model 45

✅ Submission file saved as submission.csv
